In [25]:
import pandas as pd

In [64]:
# Abrir o csv, pegar uns 20% pra teste e 60% pra treino, o resto deixa no dataset não rotulado

CSV_PATH = '../dataset/NoThemeTweets.csv'

df = pd.read_csv(CSV_PATH)
df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [67]:
df.shape

(392907, 2)

In [50]:
df = df[df['Score'] != 3]
df['sentiment'] = df['Score'].apply(lambda rating : 1 if rating > 3 else 0)

In [66]:
df.rename(columns={'tweet_text':'text','sentiment': 'label'}, inplace=True)
df = df[['text', 'label']]
df = df.sample(frac = 0.5, random_state=42)
df.head()

,text,label
193377,eu to muito gordo :(,Negativo
602080,"@nelsonfsilva8 actually same, continuo a mesma...",Positivo
468421,meu vizinho pagou 600 reais num doguinho de ra...,Negativo
448164,falar sobre política separa as pessoas :( fiqu...,Negativo
190317,Jogo grátis STEAM: For Honor Está grátis por t...,Positivo


In [68]:
# Cria dataset de treino com 60% dos valores
df_train = df.sample(frac = 0.6)

# Cria dataset com o 40% restante
df_rem = df.drop(df_train.index)

# Cria dataset de teste com metade do de treino
df_test = df_rem.sample(frac = 0.5)

# Cria dataset de unlabel com o restante
df_unlabel = df_rem.drop(df_test.index)

In [4]:
!pip install pyarrow


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip3 install --upgrade pip
Reshimming asdf python...


In [69]:
# Salva-se assim df.to_parquet("df_test.parquet.gzip", engine="pyarrow", compression="gzip")

df_train.to_parquet('../dataset/df_train.parquet.gzip', engine='pyarrow', compression='gzip')
df_test.to_parquet('../dataset/df_test.parquet.gzip', engine='pyarrow', compression='gzip')
df_unlabel.to_parquet('../dataset/df_unlabel.parquet.gzip', engine='pyarrow', compression='gzip')

In [71]:
train = pd.read_parquet('../dataset/df_train.parquet.gzip')
test = pd.read_parquet('../dataset/df_test.parquet.gzip')
unlabel = pd.read_parquet('../dataset/df_unlabeled.parquet.gzip')

unlabel.shape

(78581, 2)

## Teste modelo

In [1]:
import json
import os
import pickle
import time
from typing import Any, TypeAlias

import nltk
import pandas as pd
import requests
import schedule
from modAL.batch import uncertainty_batch_sampling
from modAL.models.learners import ActiveLearner
from scipy.sparse._csr import csr_matrix
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

In [2]:
def load(name: str):
        """
        Carrega qualquer tipo de dado
        """

        path = os.path.join("../dataset", name)
        path += ".pkl"
        return pickle.load(open(path, "rb"))

In [3]:
def read_parquet(name: str) -> pd.DataFrame:
        """
        Lê um parquet e o retorna em DataFrame
        """

        path = os.path.join("../dataset", name)
        path += ".parquet.gzip"
        return pd.read_parquet(path, engine="pyarrow")

In [4]:
def data_preprocessing(df: pd.DataFrame):
        """
        Pré-processamento dos dados, retornando dados vetorizados e seus rótulos
        """
        print(df.head())
        X = vectorizer.transform(df["text"])
        Y = le.transform(df["label"])
        # Y = self.le.transform(df["class"])
        return X, Y

In [5]:
print("Stopwords")
stopwords = nltk.corpus.stopwords.words("portuguese")

# Vectorizer
print("Vectorizer")
vectorizer = TfidfVectorizer(
    stop_words=stopwords, max_df=0.90, min_df=0.15
)
df_unlabeled = read_parquet("df_unlabeled")

print("Label Encoder")
# classes = json.load(open("data/classes.json", "r"))
le = preprocessing.LabelEncoder()
# le.fit(classes)
le.fit(["Positivo", "Negativo"])
# le.fit(["cacm", "cisi", "cran", "med"])

vectorizer.fit(df_unlabeled["text"])

print("Train")
df_train = read_parquet("df_train")
x_train, y_train = data_preprocessing(df_train)

Stopwords
Vectorizer
Label Encoder
Train
                                                     text     label
102141             @luvv29 migo é tão bom ouvir isso :(((  Negativo
330185  @JH94W EU VOLTEI P WENDY ESSA SEMANA N DESISTE...  Negativo
431988  @lolasmagicshop brigadaaaaaaaaaaa teee amooooo...  Negativo
498948           amanhã tem show e o rugg tá com febre :(  Negativo
143176  @Thai__Helena @CNCOmusic O problema é eu quere...  Negativo


In [8]:
# create a XGBClassifier
print("XGBClassifier")
xgb = XGBClassifier()
xgb.fit(x_train, y_train)


XGBClassifier


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [19]:
# import linearRegression from sklearn
from sklearn.linear_model import LinearRegression
# create a LinearRegression
print("LinearRegression")
lr = LinearRegression()
lr.fit(x_train, y_train)


LinearRegression


LinearRegression()

In [20]:
lr.predict(vec_text)

array([0.32067753, 0.32067753, 0.32067753, 0.39375498, 0.32067753,
       0.32067753, 0.32067753, 0.32067753, 0.32067753, 0.32067753])

In [9]:
df_unlabeled.iloc[0]

text     @Tixaa23 14 para eu ir :)
label                     Positivo
Name: 0, dtype: object

In [13]:
vec_text = vectorizer.transform(df_unlabeled.iloc[0:10]["text"])
xgb.predict(vec_text)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [14]:
df_unlabeled.iloc[0:10]["label"]

0     Positivo
4     Positivo
5     Positivo
18    Positivo
19    Positivo
21    Positivo
22    Positivo
26    Positivo
27    Positivo
31    Positivo
Name: label, dtype: object

In [61]:
xgb.predict_proba(vec_text)[0].max() - 0.5

array([[0.6793245, 0.3206755],
       [0.6793245, 0.3206755],
       [0.6793245, 0.3206755],
       [0.6062312, 0.3937688],
       [0.6793245, 0.3206755],
       [0.6793245, 0.3206755],
       [0.6793245, 0.3206755],
       [0.6793245, 0.3206755],
       [0.6793245, 0.3206755],
       [0.6793245, 0.3206755]], dtype=float32)

In [62]:
xgb.predict(vec_text)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
df_unlabeled.iloc[0:10]["text"]

0                             @Tixaa23 14 para eu ir :)
4     @Primo_Resmungao Pq da pr jeito!!é uma "oferta...
5     @BadWolf_Wagner @DanieVedo Eu entendi, mas iss...
18    Gostei de um vídeo @YouTube https://t.co/FGEgH...
19    Nenhum dos itens são meus. Tô só fazendo aquel...
21    @barbaraveronni1 @anacxrvalho @Just_Foxyz. sim...
22    Não estou conseguindo twittar muito pois meu c...
26         ia marcar a ceci mas ela me deu softblock :)
27                   @aclarapinheir0 Delicada demais :D
31            @julialimapessoa Boa noite, Pessoinha :))
Name: text, dtype: object

In [21]:
le.classes_

array(['Negativo', 'Positivo'], dtype='<U8')

In [24]:
# get dictionary of classes
classes = le.classes_.tolist()
classes

['Negativo', 'Positivo']